# EDA con Enfoque en CAC y LTV

Este cuaderno se centra en **preguntas de negocio** que generan valor al entender la relación entre:
- **CAC**: Costo de Adquisición de Cliente.
- **LTV**: Valor de Vida del Cliente.

Adicionalmente, explora aspectos clave como **suscripción**, **descuentos**, **categoría** del producto, **frecuencia de compras**, **método de pago**, etc. Con estas preguntas podrás tomar **decisiones estratégicas** sobre tu modelo de negocio, campañas de marketing y retención de clientes.

## Contenido:
1. **Importar librerías y cargar datos**
2. **Inspección inicial** (dimensiones, tipos de datos, nulos)
3. **Limpieza básica** (si es necesaria)
4. **Creación de columna** ratio `LTV/CAC` (rentabilidad por cliente)
5. **Preguntas de negocio** (groupby, filtros, etc.)
6. **Conclusiones**

In [1]:
!wget !wget https://github.com/javierherrera1996/IntroMarketingAnalytics/raw/refs/heads/main/PrimerCorte/shopping_trends_LTV_CAC.csv

--2025-03-12 21:27:25--  http://!wget/
Resolving !wget (!wget)... failed: Name or service not known.
wget: unable to resolve host address ‘!wget’
--2025-03-12 21:27:25--  https://github.com/javierherrera1996/IntroMarketingAnalytics/raw/refs/heads/main/PrimerCorte/shopping_trends_LTV_CAC.csv
Resolving github.com (github.com)... 140.82.113.3
Connecting to github.com (github.com)|140.82.113.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/javierherrera1996/IntroMarketingAnalytics/refs/heads/main/PrimerCorte/shopping_trends_LTV_CAC.csv [following]
--2025-03-12 21:27:25--  https://raw.githubusercontent.com/javierherrera1996/IntroMarketingAnalytics/refs/heads/main/PrimerCorte/shopping_trends_LTV_CAC.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HT

## 1. Importar librerías y cargar datos
Asegúrate de tener tu archivo CSV con las columnas mencionadas: `CAC`, `LTV`, `Purchase Amount (USD)`, etc. Ajusta la ruta según tu archivo real.

In [2]:
import pandas as pd
import numpy as np


# Cargar el dataset
df = pd.read_csv("shopping_trends_LTV_CAC.csv")


# Mostramos las primeras filas para verificar
df.head()

,Customer ID,Age,Gender,Item Purchased,Category,Purchase Amount (USD),Location,Size,Color,Season,...,Shipping Type,Discount Applied,Promo Code Used,Previous Purchases,Preferred Payment Method,Frequency of Purchases,CAC,LTV,Rentabilidad,LTV_CAC_Ratio
0,1,55,Male,Blouse,Clothing,53,Kentucky,L,Gray,Winter,...,Express,Yes,Yes,14,Venmo,Fortnightly,112.0,1512,16.434783,13.500000
1,2,19,Male,Sweater,Clothing,64,Maine,L,Maroon,Winter,...,Express,Yes,Yes,2,Cash,Fortnightly,95.0,1497,19.960000,15.757895
2,3,50,Male,Jeans,Clothing,73,Massachusetts,S,Maroon,Spring,...,Free Shipping,Yes,Yes,23,Credit Card,Weekly,32.0,1137,94.750000,35.531250
3,4,21,Male,Sandals,Footwear,90,Rhode Island,M,Maroon,Spring,...,Next Day Air,Yes,Yes,49,PayPal,Weekly,NaN,1914,26.957746,26.957746
4,5,45,Male,Blouse,Clothing,49,Oregon,M,Turquoise,Spring,...,Free Shipping,Yes,Yes,31,PayPal,Annually,47.0,1300,48.148148,27.659574


## 2. Inspección inicial
1. **Dimensiones (filas x columnas)**
2. **Info general**: tipos de dato, nulos, etc.
3. **Valores nulos** por columna.

Si existen valores nulos en columnas importantes (por ejemplo, `CAC`, `LTV`, `Customer ID`), evalúa si los eliminarás o imputarás.

In [8]:
"Dimensiones"
df.shape

(3900, 23)

In [7]:
"Información general"
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3900 entries, 0 to 3899
Data columns (total 23 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Customer ID               3900 non-null   int64  
 1   Age                       3900 non-null   int64  
 2   Gender                    3900 non-null   object 
 3   Item Purchased            3900 non-null   object 
 4   Category                  3900 non-null   object 
 5   Purchase Amount (USD)     3900 non-null   int64  
 6   Location                  3900 non-null   object 
 7   Size                      3900 non-null   object 
 8   Color                     3900 non-null   object 
 9   Season                    3900 non-null   object 
 10  Review Rating             3900 non-null   float64
 11  Subscription Status       3900 non-null   object 
 12  Payment Method            3900 non-null   object 
 13  Shipping Type             3900 non-null   object 
 14  Discount

In [6]:
"valores nulos por columna"
df.isnull().sum()

,0
Customer ID,0
Age,0
Gender,0
Item Purchased,0
Category,0
Purchase Amount (USD),0
Location,0
Size,0
Color,0
Season,0


## 3. Limpieza básica
Por ejemplo, si detectas `CAC <= 0` o `LTV <= 0` y consideras que no tienen sentido en tu negocio, podrías filtrar esos registros. Ajusta según tu criterio.

In [11]:
df = df[(df['CAC'] > 0) & (df['LTV'] > 0)]
df.head()

,Customer ID,Age,Gender,Item Purchased,Category,Purchase Amount (USD),Location,Size,Color,Season,...,Shipping Type,Discount Applied,Promo Code Used,Previous Purchases,Preferred Payment Method,Frequency of Purchases,CAC,LTV,Rentabilidad,LTV_CAC_Ratio
0,1,55,Male,Blouse,Clothing,53,Kentucky,L,Gray,Winter,...,Express,Yes,Yes,14,Venmo,Fortnightly,112.0,1512,16.434783,13.500000
1,2,19,Male,Sweater,Clothing,64,Maine,L,Maroon,Winter,...,Express,Yes,Yes,2,Cash,Fortnightly,95.0,1497,19.960000,15.757895
2,3,50,Male,Jeans,Clothing,73,Massachusetts,S,Maroon,Spring,...,Free Shipping,Yes,Yes,23,Credit Card,Weekly,32.0,1137,94.750000,35.531250
4,5,45,Male,Blouse,Clothing,49,Oregon,M,Turquoise,Spring,...,Free Shipping,Yes,Yes,31,PayPal,Annually,47.0,1300,48.148148,27.659574
6,7,63,Male,Shirt,Clothing,85,Montana,M,Gray,Fall,...,Free Shipping,Yes,Yes,49,Cash,Quarterly,99.0,213,2.696203,2.151515


## 4. Creación de columna: Ratio LTV/CAC
Es una métrica clave de la **rentabilidad** del cliente. Un valor > 1 indica que, en teoría, el cliente retorna más de lo que cuesta adquirirlo.

In [13]:
df['LTV_CAC_Ratio'] = df['LTV'] / df['CAC']
df.head()


,Customer ID,Age,Gender,Item Purchased,Category,Purchase Amount (USD),Location,Size,Color,Season,...,Shipping Type,Discount Applied,Promo Code Used,Previous Purchases,Preferred Payment Method,Frequency of Purchases,CAC,LTV,Rentabilidad,LTV_CAC_Ratio
0,1,55,Male,Blouse,Clothing,53,Kentucky,L,Gray,Winter,...,Express,Yes,Yes,14,Venmo,Fortnightly,112.0,1512,16.434783,13.500000
1,2,19,Male,Sweater,Clothing,64,Maine,L,Maroon,Winter,...,Express,Yes,Yes,2,Cash,Fortnightly,95.0,1497,19.960000,15.757895
2,3,50,Male,Jeans,Clothing,73,Massachusetts,S,Maroon,Spring,...,Free Shipping,Yes,Yes,23,Credit Card,Weekly,32.0,1137,94.750000,35.531250
4,5,45,Male,Blouse,Clothing,49,Oregon,M,Turquoise,Spring,...,Free Shipping,Yes,Yes,31,PayPal,Annually,47.0,1300,48.148148,27.659574
6,7,63,Male,Shirt,Clothing,85,Montana,M,Gray,Fall,...,Free Shipping,Yes,Yes,49,Cash,Quarterly,99.0,213,2.696203,2.151515


## 5. Preguntas de Negocio
En vez de usar gráficas o correlaciones, nos centramos en **preguntas que generen valor** al análisis. Cada una se resuelve con **groupby, filtros o estadísticas** simples.

1. **¿Cuántos clientes únicos hay y cuál es la edad promedio?**
2. **¿Cuál es el CAC promedio y el LTV promedio en todo el dataset?**
3. **¿Cuál es la ratio LTV/CAC promedio?**
4. **¿Qué categoría tiene el mayor LTV en promedio?**
5. **¿En cuál categoría se justifica más el CAC?** (categoría con mayor ratio LTV/CAC)
6. **¿Los clientes suscritos tienen LTV mayor que los no suscritos?**
7. **¿Cuál es el método de pago más frecuente y su LTV promedio?**
8. **¿Quiénes usan descuentos con mayor frecuencia?** (Por ejemplo, ¿cierto género, cierto rango de edad?)
9. **¿Cuántos clientes aprovechan códigos promocionales (Promo Code Used)?** y su ratio LTV/CAC
10. **¿Cuál es la frecuencia de compra más común (Frequency of Purchases)?** y su LTV asociado.
11. **¿Cuál es la ubicación (Location) con mayor ratio LTV/CAC?**
12. **Entre los clientes con `Previous Purchases` > X, cuál es el gasto promedio y LTV promedio?**


### 5.1 ¿Cuántos clientes únicos hay y cuál es la edad promedio?

In [15]:
clientes_unicos = df['Customer ID'].nunique()
edad_promedio = df['Age'].mean()
df.head()

,Customer ID,Age,Gender,Item Purchased,Category,Purchase Amount (USD),Location,Size,Color,Season,...,Shipping Type,Discount Applied,Promo Code Used,Previous Purchases,Preferred Payment Method,Frequency of Purchases,CAC,LTV,Rentabilidad,LTV_CAC_Ratio
0,1,55,Male,Blouse,Clothing,53,Kentucky,L,Gray,Winter,...,Express,Yes,Yes,14,Venmo,Fortnightly,112.0,1512,16.434783,13.500000
1,2,19,Male,Sweater,Clothing,64,Maine,L,Maroon,Winter,...,Express,Yes,Yes,2,Cash,Fortnightly,95.0,1497,19.960000,15.757895
2,3,50,Male,Jeans,Clothing,73,Massachusetts,S,Maroon,Spring,...,Free Shipping,Yes,Yes,23,Credit Card,Weekly,32.0,1137,94.750000,35.531250
4,5,45,Male,Blouse,Clothing,49,Oregon,M,Turquoise,Spring,...,Free Shipping,Yes,Yes,31,PayPal,Annually,47.0,1300,48.148148,27.659574
6,7,63,Male,Shirt,Clothing,85,Montana,M,Gray,Fall,...,Free Shipping,Yes,Yes,49,Cash,Quarterly,99.0,213,2.696203,2.151515


### 5.2 ¿Cuál es el CAC promedio y el LTV promedio en todo el dataset?

In [16]:
cac_promedio = df['CAC'].mean()
ltv_promedio = df['LTV'].mean()
df.head()

,Customer ID,Age,Gender,Item Purchased,Category,Purchase Amount (USD),Location,Size,Color,Season,...,Shipping Type,Discount Applied,Promo Code Used,Previous Purchases,Preferred Payment Method,Frequency of Purchases,CAC,LTV,Rentabilidad,LTV_CAC_Ratio
0,1,55,Male,Blouse,Clothing,53,Kentucky,L,Gray,Winter,...,Express,Yes,Yes,14,Venmo,Fortnightly,112.0,1512,16.434783,13.500000
1,2,19,Male,Sweater,Clothing,64,Maine,L,Maroon,Winter,...,Express,Yes,Yes,2,Cash,Fortnightly,95.0,1497,19.960000,15.757895
2,3,50,Male,Jeans,Clothing,73,Massachusetts,S,Maroon,Spring,...,Free Shipping,Yes,Yes,23,Credit Card,Weekly,32.0,1137,94.750000,35.531250
4,5,45,Male,Blouse,Clothing,49,Oregon,M,Turquoise,Spring,...,Free Shipping,Yes,Yes,31,PayPal,Annually,47.0,1300,48.148148,27.659574
6,7,63,Male,Shirt,Clothing,85,Montana,M,Gray,Fall,...,Free Shipping,Yes,Yes,49,Cash,Quarterly,99.0,213,2.696203,2.151515


### 5.3 ¿Cuál es la ratio LTV/CAC promedio?

In [17]:
ratio_promedio = df['LTV_CAC_Ratio'].mean()
df.head()

,Customer ID,Age,Gender,Item Purchased,Category,Purchase Amount (USD),Location,Size,Color,Season,...,Shipping Type,Discount Applied,Promo Code Used,Previous Purchases,Preferred Payment Method,Frequency of Purchases,CAC,LTV,Rentabilidad,LTV_CAC_Ratio
0,1,55,Male,Blouse,Clothing,53,Kentucky,L,Gray,Winter,...,Express,Yes,Yes,14,Venmo,Fortnightly,112.0,1512,16.434783,13.500000
1,2,19,Male,Sweater,Clothing,64,Maine,L,Maroon,Winter,...,Express,Yes,Yes,2,Cash,Fortnightly,95.0,1497,19.960000,15.757895
2,3,50,Male,Jeans,Clothing,73,Massachusetts,S,Maroon,Spring,...,Free Shipping,Yes,Yes,23,Credit Card,Weekly,32.0,1137,94.750000,35.531250
4,5,45,Male,Blouse,Clothing,49,Oregon,M,Turquoise,Spring,...,Free Shipping,Yes,Yes,31,PayPal,Annually,47.0,1300,48.148148,27.659574
6,7,63,Male,Shirt,Clothing,85,Montana,M,Gray,Fall,...,Free Shipping,Yes,Yes,49,Cash,Quarterly,99.0,213,2.696203,2.151515


### 5.4 ¿Qué categoría tiene el mayor LTV en promedio?

In [18]:
categoria_mayor_ltv = df.groupby('Category')['LTV'].mean().idxmax()
df.head()

,Customer ID,Age,Gender,Item Purchased,Category,Purchase Amount (USD),Location,Size,Color,Season,...,Shipping Type,Discount Applied,Promo Code Used,Previous Purchases,Preferred Payment Method,Frequency of Purchases,CAC,LTV,Rentabilidad,LTV_CAC_Ratio
0,1,55,Male,Blouse,Clothing,53,Kentucky,L,Gray,Winter,...,Express,Yes,Yes,14,Venmo,Fortnightly,112.0,1512,16.434783,13.500000
1,2,19,Male,Sweater,Clothing,64,Maine,L,Maroon,Winter,...,Express,Yes,Yes,2,Cash,Fortnightly,95.0,1497,19.960000,15.757895
2,3,50,Male,Jeans,Clothing,73,Massachusetts,S,Maroon,Spring,...,Free Shipping,Yes,Yes,23,Credit Card,Weekly,32.0,1137,94.750000,35.531250
4,5,45,Male,Blouse,Clothing,49,Oregon,M,Turquoise,Spring,...,Free Shipping,Yes,Yes,31,PayPal,Annually,47.0,1300,48.148148,27.659574
6,7,63,Male,Shirt,Clothing,85,Montana,M,Gray,Fall,...,Free Shipping,Yes,Yes,49,Cash,Quarterly,99.0,213,2.696203,2.151515


### 5.5 ¿En cuál categoría se justifica más el CAC? (mayor ratio LTV/CAC)

In [19]:
categoria_justifica_cac = df.groupby('Category')['LTV_CAC_Ratio'].mean().idxmax()
df.head()

,Customer ID,Age,Gender,Item Purchased,Category,Purchase Amount (USD),Location,Size,Color,Season,...,Shipping Type,Discount Applied,Promo Code Used,Previous Purchases,Preferred Payment Method,Frequency of Purchases,CAC,LTV,Rentabilidad,LTV_CAC_Ratio
0,1,55,Male,Blouse,Clothing,53,Kentucky,L,Gray,Winter,...,Express,Yes,Yes,14,Venmo,Fortnightly,112.0,1512,16.434783,13.500000
1,2,19,Male,Sweater,Clothing,64,Maine,L,Maroon,Winter,...,Express,Yes,Yes,2,Cash,Fortnightly,95.0,1497,19.960000,15.757895
2,3,50,Male,Jeans,Clothing,73,Massachusetts,S,Maroon,Spring,...,Free Shipping,Yes,Yes,23,Credit Card,Weekly,32.0,1137,94.750000,35.531250
4,5,45,Male,Blouse,Clothing,49,Oregon,M,Turquoise,Spring,...,Free Shipping,Yes,Yes,31,PayPal,Annually,47.0,1300,48.148148,27.659574
6,7,63,Male,Shirt,Clothing,85,Montana,M,Gray,Fall,...,Free Shipping,Yes,Yes,49,Cash,Quarterly,99.0,213,2.696203,2.151515


### 5.6 ¿Los clientes suscritos tienen LTV mayor que los no suscritos?

In [37]:
df.groupby('Subscription Status')['LTV'].mean()
df.head()


,Customer ID,Age,Gender,Item Purchased,Category,Purchase Amount (USD),Location,Size,Color,Season,...,Shipping Type,Discount Applied,Promo Code Used,Previous Purchases,Preferred Payment Method,Frequency of Purchases,CAC,LTV,Rentabilidad,LTV_CAC_Ratio
0,1,55,Male,Blouse,Clothing,53,Kentucky,L,Gray,Winter,...,Express,Yes,Yes,14,Venmo,Fortnightly,112.0,1512,16.434783,13.500000
1,2,19,Male,Sweater,Clothing,64,Maine,L,Maroon,Winter,...,Express,Yes,Yes,2,Cash,Fortnightly,95.0,1497,19.960000,15.757895
2,3,50,Male,Jeans,Clothing,73,Massachusetts,S,Maroon,Spring,...,Free Shipping,Yes,Yes,23,Credit Card,Weekly,32.0,1137,94.750000,35.531250
4,5,45,Male,Blouse,Clothing,49,Oregon,M,Turquoise,Spring,...,Free Shipping,Yes,Yes,31,PayPal,Annually,47.0,1300,48.148148,27.659574
6,7,63,Male,Shirt,Clothing,85,Montana,M,Gray,Fall,...,Free Shipping,Yes,Yes,49,Cash,Quarterly,99.0,213,2.696203,2.151515


### 5.7 ¿Cuál es el método de pago más frecuente y su LTV promedio?

In [26]:
ltv_promedio_metodo = df.groupby('Payment Method')['LTV'].mean()
df.head()

,Customer ID,Age,Gender,Item Purchased,Category,Purchase Amount (USD),Location,Size,Color,Season,...,Shipping Type,Discount Applied,Promo Code Used,Previous Purchases,Preferred Payment Method,Frequency of Purchases,CAC,LTV,Rentabilidad,LTV_CAC_Ratio
0,1,55,Male,Blouse,Clothing,53,Kentucky,L,Gray,Winter,...,Express,Yes,Yes,14,Venmo,Fortnightly,112.0,1512,16.434783,13.500000
1,2,19,Male,Sweater,Clothing,64,Maine,L,Maroon,Winter,...,Express,Yes,Yes,2,Cash,Fortnightly,95.0,1497,19.960000,15.757895
2,3,50,Male,Jeans,Clothing,73,Massachusetts,S,Maroon,Spring,...,Free Shipping,Yes,Yes,23,Credit Card,Weekly,32.0,1137,94.750000,35.531250
4,5,45,Male,Blouse,Clothing,49,Oregon,M,Turquoise,Spring,...,Free Shipping,Yes,Yes,31,PayPal,Annually,47.0,1300,48.148148,27.659574
6,7,63,Male,Shirt,Clothing,85,Montana,M,Gray,Fall,...,Free Shipping,Yes,Yes,49,Cash,Quarterly,99.0,213,2.696203,2.151515


### 5.8 ¿Quiénes usan descuentos con mayor frecuencia?
Podemos ver si hay algún grupo (por género, rango de edad, etc.) que destaca. A modo de ejemplo, segmentaremos por género:

In [48]:
df['Discount Applied'] = pd.to_numeric(df['Discount Applied'], errors='coerce').fillna(0)
df.groupby('Gender')['Discount Applied'].mean()
df.groupby(pd.cut(df['Age'], bins=[18, 25, 35, 50, 65, 100]))['Discount Applied'].mean()



<ipython-input-48-fe4f4d8cacef>:3: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df.groupby(pd.cut(df['Age'], bins=[18, 25, 35, 50, 65, 100]))['Discount Applied'].mean()


,Discount Applied
Age,
"(18, 25]",0.0
"(25, 35]",0.0
"(35, 50]",0.0
"(50, 65]",0.0
"(65, 100]",0.0


Podrías repetir el ejercicio segmentando por **Age** (definiendo rangos) o por **Location**.

### 5.9 ¿Cuántos clientes aprovechan códigos promocionales y cuál es su ratio LTV/CAC?
El uso de promos puede modificar la rentabilidad.

In [45]:
df['Promo Code Used'] = pd.to_numeric(df['Promo Code Used'], errors='coerce').fillna(0)

promo_cliente = df[df['Promo Code Used'] > 0]['Customer ID'].nunique()
LTV_CAC = df[df['Promo Code Used'] > 0]['LTV_CAC_Ratio'].mean()

df.head()


,Customer ID,Age,Gender,Item Purchased,Category,Purchase Amount (USD),Location,Size,Color,Season,...,Shipping Type,Discount Applied,Promo Code Used,Previous Purchases,Preferred Payment Method,Frequency of Purchases,CAC,LTV,Rentabilidad,LTV_CAC_Ratio
0,1,55,Male,Blouse,Clothing,53,Kentucky,L,Gray,Winter,...,Express,Yes,0.0,14,Venmo,Fortnightly,112.0,1512,16.434783,13.500000
1,2,19,Male,Sweater,Clothing,64,Maine,L,Maroon,Winter,...,Express,Yes,0.0,2,Cash,Fortnightly,95.0,1497,19.960000,15.757895
2,3,50,Male,Jeans,Clothing,73,Massachusetts,S,Maroon,Spring,...,Free Shipping,Yes,0.0,23,Credit Card,Weekly,32.0,1137,94.750000,35.531250
4,5,45,Male,Blouse,Clothing,49,Oregon,M,Turquoise,Spring,...,Free Shipping,Yes,0.0,31,PayPal,Annually,47.0,1300,48.148148,27.659574
6,7,63,Male,Shirt,Clothing,85,Montana,M,Gray,Fall,...,Free Shipping,Yes,0.0,49,Cash,Quarterly,99.0,213,2.696203,2.151515


### 5.10 ¿Cuál es la frecuencia de compra más común (Frequency of Purchases) y su LTV?
Esto ayuda a ver si, por ejemplo, los clientes que compran semanalmente tienen un LTV más alto.

In [32]:
frecuencia_mas_comun = df['Frequency of Purchases'].mode()[0]
ltv_frecuencia = df.groupby('Frequency of Purchases')['LTV'].mean()
df.head()

,Customer ID,Age,Gender,Item Purchased,Category,Purchase Amount (USD),Location,Size,Color,Season,...,Shipping Type,Discount Applied,Promo Code Used,Previous Purchases,Preferred Payment Method,Frequency of Purchases,CAC,LTV,Rentabilidad,LTV_CAC_Ratio
0,1,55,Male,Blouse,Clothing,53,Kentucky,L,Gray,Winter,...,Express,Yes,Yes,14,Venmo,Fortnightly,112.0,1512,16.434783,13.500000
1,2,19,Male,Sweater,Clothing,64,Maine,L,Maroon,Winter,...,Express,Yes,Yes,2,Cash,Fortnightly,95.0,1497,19.960000,15.757895
2,3,50,Male,Jeans,Clothing,73,Massachusetts,S,Maroon,Spring,...,Free Shipping,Yes,Yes,23,Credit Card,Weekly,32.0,1137,94.750000,35.531250
4,5,45,Male,Blouse,Clothing,49,Oregon,M,Turquoise,Spring,...,Free Shipping,Yes,Yes,31,PayPal,Annually,47.0,1300,48.148148,27.659574
6,7,63,Male,Shirt,Clothing,85,Montana,M,Gray,Fall,...,Free Shipping,Yes,Yes,49,Cash,Quarterly,99.0,213,2.696203,2.151515


### 5.11 ¿Cuál es la ubicación (Location) con mayor ratio LTV/CAC?

In [35]:
ubicacion_max_ratio = df.groupby('Location')['LTV_CAC_Ratio'].mean().idxmax()
df.head()

,Customer ID,Age,Gender,Item Purchased,Category,Purchase Amount (USD),Location,Size,Color,Season,...,Shipping Type,Discount Applied,Promo Code Used,Previous Purchases,Preferred Payment Method,Frequency of Purchases,CAC,LTV,Rentabilidad,LTV_CAC_Ratio
0,1,55,Male,Blouse,Clothing,53,Kentucky,L,Gray,Winter,...,Express,Yes,Yes,14,Venmo,Fortnightly,112.0,1512,16.434783,13.500000
1,2,19,Male,Sweater,Clothing,64,Maine,L,Maroon,Winter,...,Express,Yes,Yes,2,Cash,Fortnightly,95.0,1497,19.960000,15.757895
2,3,50,Male,Jeans,Clothing,73,Massachusetts,S,Maroon,Spring,...,Free Shipping,Yes,Yes,23,Credit Card,Weekly,32.0,1137,94.750000,35.531250
4,5,45,Male,Blouse,Clothing,49,Oregon,M,Turquoise,Spring,...,Free Shipping,Yes,Yes,31,PayPal,Annually,47.0,1300,48.148148,27.659574
6,7,63,Male,Shirt,Clothing,85,Montana,M,Gray,Fall,...,Free Shipping,Yes,Yes,49,Cash,Quarterly,99.0,213,2.696203,2.151515


### 5.12 Entre los clientes con `Previous Purchases` > X, ¿cuál es el gasto promedio y el LTV promedio?
Define un umbral X según tu contexto. Por ejemplo, X = 20:

In [52]:
X = 20
df['Previous Purchases'] = pd.to_numeric(df['Previous Purchases'], errors='coerce').fillna(0)
df_filtrado = df[df['Previous Purchases'] > X]
if not df_filtrado.empty:
    if 'Purchase Amount (USD)' in df_filtrado.columns:
        gasto_promedio = df_filtrado['Purchase Amount (USD)'].fillna(0).mean()
    else:
        print("Error: Column 'Spend' or 'Purchase Amount (USD)' not found in DataFrame.")
        gasto_promedio = 0

    ltv_promedio = df_filtrado['LTV'].fillna(0).mean()
else:
    gasto_promedio = 0
    ltv_promedio = 0

print(f"Gasto promedio: {gasto_promedio}")
print(f"LTV promedio: {ltv_promedio}")

Gasto promedio: 59.63687150837989
LTV promedio: 1121.810563737938


## 6. Conclusiones

En este Notebook, hemos abordado un **EDA orientado a negocio** en el que:
1. **Identificamos** la base total de clientes y su edad promedio.
2. **Calculamos** el CAC y LTV promedio, y la ratio LTV/CAC global.
3. **Segregamos** por categorías, suscripción, uso de descuentos, métodos de pago, etc.
4. **Resolvimos** preguntas de negocio que ayudan a:
   - **Identificar** cuáles categorías o ubicaciones ofrecen mejor rentabilidad (ratio LTV/CAC).
   - **Entender** si la suscripción eleva el LTV.
   - **Explorar** la efectividad de promos y descuentos.
   - **Diferenciar** frecuencias de compra y ver su impacto en LTV.
   - **Focalizar** inversiones de marketing en los segmentos más prometedores.

Este tipo de **análisis** proporciona las bases para **estrategias** de retención, adquisición y segmentación de clientes. ¡Espero que te sea útil para **generar valor** en tu negocio!